https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

https://learn.deeplearning.ai/courses/knowledge-graphs-rag/lesson/5/constructing-a-knowledge-graph-from-text-documents

https://learn.deeplearning.ai/courses/advanced-retrieval-for-ai/lesson/1/introduction

In [1]:
from embedding_model import Embedder
import chromadb




In [2]:
embedder = Embedder(model_name='sentence-transformers/all-MiniLM-L12-v2',
                    tokenizer_name='sentence-transformers/all-MiniLM-L12-v2')

2024-03-15 12:07:30.871025: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 12:07:32.848799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Chroma DB

In [13]:
!pip3 show chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Name: chromadb
Version: 0.4.9
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /home/stanislav/.local/lib/python3.10/site-packages
Requires: bcrypt, chroma-hnswlib, fastapi, importlib-resources, numpy, onnxruntime, overrides, posthog, pulsar-client, pydantic, pypika, requests, tokenizers, tqdm, typing-extensions, uvicorn
Required-by: 


In [7]:


db_client = chromadb.PersistentClient(path="./persistent_storage")

collection = db_client.get_collection("my_collection")



In [36]:
# ##Adding docs

# collection.add(
    
    
#     documents=docs,
#     metadatas=[{"len" : len(docs[0]), "uri" : 232},
#                {"len" : len(docs[1]), "uri" : 235},
#               {"len" : len(docs[2]), "uri" : 239}],
    
#    ids=["uri9", "uri10", "uri12"],
    
#     embeddings=[ 
#                 [0.1, 0.3, 0.4],
#                  [-0.1, 0.3, 0.5],
#                  [2, 0, 0.03]  
#                             ]
        
# )

In [10]:
collection.peek(2)

{'ids': ['uri9', 'uri10'],
 'embeddings': [[0.10000000149011612, 0.30000001192092896, 0.4000000059604645],
  [-0.10000000149011612, 0.30000001192092896, 0.5]],
 'metadatas': [{'len': 28, 'uri': 232}, {'len': 35, 'uri': 235}],
 'documents': ['Hello World! My name is Stas',
  'Let me introduce my self, I am Stas']}

In [11]:
collection.query([[2, 0.27, 0.33],],
                 n_results=2)

{'ids': [['uri12', 'uri9']],
 'distances': [[0.16290000040233138, 3.6157999958872797]],
 'metadatas': [[{'len': 40, 'uri': 239}, {'len': 28, 'uri': 232}]],
 'embeddings': None,
 'documents': [['A white cat has eaten and now he is full',
   'Hello World! My name is Stas']]}

### Rough plan 

ChromaDB + TextEmbedding + ChatGPT3 + Custom Promts and chains + Web UI (optional)



- application takes path to directory with pdfs as argument
- split docs into chunks + add metadata
- compute embeddings and put chunks to Chroma
- Create chains for Q&A that uses metada and refers to uris of docs
- put app to Docker container
- Query expansion + Cross-Encoder re-ranking

## OpenAI API

In [31]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [33]:



client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Make up a poem about Deep Learning."}
  ]
)

In [34]:
print(completion.choices[0].message.content)

In the realm of data and of code,
Where machines learn to take the load,
Deep Learning rises like a star,
Unfolding patterns near and far.

Hidden layers, neurons fire,
Mapping inputs to desired higher,
Through iterations and backpropagation,
It refines its own creation.

Images, text, and sounds it sees,
Extracting meaning with ease,
Pattern recognition never dull,
In the deep, it finds the pull.

Complex tasks it can embrace,
Unraveling data's intricate maze,
With each epoch, it grows stronger,
A digital mind, lasting longer.

So let us marvel at this wonder,
Of algorithms tearing asunder,
The boundaries of what we know,
Deep Learning, in its brilliance, shows.
